# Loopy IPython Integration Demo

In [1]:
%load_ext loopy.ipython_ext

## Without transform code

In [2]:
%%fortran_kernel

subroutine fill(out, a, n)
  implicit none

  real*8 a, out(n)
  integer n, i

  do i = 1, n
    out(i) = a
  end do
end

/home/kaushikggg/pack/loopy_kc_env/src/loopy/loopy/frontend/fortran/translator.py:807: LoopyWarning: 'lang_version' was not passed to make_function(). To avoid this warning, pass lang_version=(2018, 2) in this invocation. (Or say 'from loopy.version import LOOPY_USE_LANGUAGE_VERSION_2018_2' in the global scope of the calling frame.)
  seq_dependencies=seq_dependencies,


In [3]:
print(prog)

---------------------------------------------------------------------------
KERNEL: fill
---------------------------------------------------------------------------
ARGUMENTS:
a: ValueArg, type: np:dtype('float64')
n: ValueArg, type: np:dtype('int32')
out: type: np:dtype('float64'), shape: (n), dim_tags: (N0:stride:1) aspace: global
---------------------------------------------------------------------------
DOMAINS:
[n] -> { [i] : 0 <= i < n }
---------------------------------------------------------------------------
INAME IMPLEMENTATION TAGS:
i: None
---------------------------------------------------------------------------
INSTRUCTIONS:
for i
  out[i] = a  {id=insn0}
end i
---------------------------------------------------------------------------


## With transform code

In [4]:
split_amount = 128

In [8]:
%%transformed_fortran_kernel

subroutine tr_fill(out, a, n)
  implicit none

  real*8 a, out(n)
  integer n, i

  do i = 1, n
    out(i) = a
  end do
end

!$loopy begin
!
! tr_fill = lp.parse_fortran(SOURCE)
! tr_fill = lp.split_iname(tr_fill, "i", split_amount,
!     outer_tag="g.0", inner_tag="l.0")
! RESULT = tr_fill
!
!$loopy end

In [10]:
print(prog)

---------------------------------------------------------------------------
KERNEL: tr_fill
---------------------------------------------------------------------------
ARGUMENTS:
a: ValueArg, type: np:dtype('float64')
n: ValueArg, type: np:dtype('int32')
out: type: np:dtype('float64'), shape: (n), dim_tags: (N0:stride:1) aspace: global
---------------------------------------------------------------------------
DOMAINS:
[n] -> { [i_outer, i_inner] : i_inner >= 0 and -128i_outer <= i_inner <= 127 and i_inner < n - 128i_outer }
---------------------------------------------------------------------------
INAME IMPLEMENTATION TAGS:
i_inner: l.0
i_outer: g.0
---------------------------------------------------------------------------
INSTRUCTIONS:
for i_inner, i_outer
    out[i_inner + i_outer*128] = a  {id=insn0}
end i_inner, i_outer
---------------------------------------------------------------------------
